# Mental Health Sentiment Analysis

In [42]:
import pandas as pd
import nltk
import torch
from datasets import Dataset
from transformers import AutoTokenizer
import numpy as np

In [19]:
df = pd.read_csv("./data/Combined Data.csv")

In [ ]:
df

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety


In [21]:
"""
Cell generated by Data Wrangler.
"""
import pandas as pd

def clean_data(df):
    # Drop column: 'Unnamed: 0'
    df = df.drop(columns=['Unnamed: 0'])
    # One-hot encode column: 'status'
    insert_loc = df.columns.get_loc('status')
    df = pd.concat([df.iloc[:,:insert_loc], pd.get_dummies(df.loc[:, ['status']]), df.iloc[:,insert_loc+1:]], axis=1)
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,statement,status_Anxiety,status_Bipolar,status_Depression,status_Normal,status_Personality disorder,status_Stress,status_Suicidal
0,oh my gosh,True,False,False,False,False,False,False
1,"trouble sleeping, confused mind, restless hear...",True,False,False,False,False,False,False
2,"All wrong, back off dear, forward doubt. Stay ...",True,False,False,False,False,False,False
3,I've shifted my focus to something else but I'...,True,False,False,False,False,False,False
4,"I'm restless and restless, it's been a month n...",True,False,False,False,False,False,False


In [ ]:
# df_clean = df_clean.drop(columns=['status_Anxiety'])

In [ ]:
df_clean

In [31]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df_clean):
    # Rename column 'status_Anxiety' to 'Anxiety'
    df_clean = df_clean.rename(columns={'status_Anxiety': 'Anxiety'})
    # Rename column 'status_Bipolar' to 'Bipolar'
    df_clean = df_clean.rename(columns={'status_Bipolar': 'Bipolar'})
    # Rename column 'status_Depression' to 'Depression'
    df_clean = df_clean.rename(columns={'status_Depression': 'Depression'})
    # Rename column 'status_Normal' to 'Normal'
    df_clean = df_clean.rename(columns={'status_Normal': 'Normal'})
    # Rename column 'status_Personality disorder' to 'Personality Disorder'
    df_clean = df_clean.rename(columns={'status_Personality disorder': 'Personality Disorder'})
    # Rename column 'status_Stress' to 'Stress'
    df_clean = df_clean.rename(columns={'status_Stress': 'Stress'})
    # Rename column 'status_Suicidal' to 'Suicidal'
    df_clean = df_clean.rename(columns={'status_Suicidal': 'Suicidal'})
    return df_clean

df_clean_1 = clean_data(df_clean.copy())
df_clean_1.head()

,statement,Anxiety,Bipolar,Depression,Normal,Personality Disorder,Stress,Suicidal
0,oh my gosh,True,False,False,False,False,False,False
1,"trouble sleeping, confused mind, restless hear...",True,False,False,False,False,False,False
2,"All wrong, back off dear, forward doubt. Stay ...",True,False,False,False,False,False,False
3,I've shifted my focus to something else but I'...,True,False,False,False,False,False,False
4,"I'm restless and restless, it's been a month n...",True,False,False,False,False,False,False


In [47]:
df_clean_1 = df_clean_1.dropna()

In [48]:
dataset = Dataset.from_pandas(df_clean_1)

In [49]:
tt_split = dataset.train_test_split(test_size=0.2, seed=42)

In [51]:
key_list = [key for key in tt_split['train'].features.keys() if key not in ['statement', '__index_level_0__']]
key_list

['Anxiety',
 'Bipolar',
 'Depression',
 'Normal',
 'Personality Disorder',
 'Stress',
 'Suicidal']

In [52]:
id2key = {idx:key for idx, key in enumerate(key_list)}
key2id = {key:idx for idx, key in enumerate(key_list)}

In [53]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [81]:
def preprocessing(textset):
    #print(len(textset['statement']))
    #print(textset)
    encoded_text = tokenizer(textset['statement'])
    labels_batch = {k: textset[k] for k in textset.keys() if k in key_list}
    labels_matrix = np.zeros((len(textset['statement']), len(key_list)))
    #print(labels_matrix.shape)
    #for label in key_list:
        #print(f"Label: {label}, Shape: {np.array(labels_batch[label]).shape}")
    for idx, label in enumerate(key_list):
        labels_matrix[:, idx] = labels_batch[label]
    encoded_text["labels"] = labels_matrix.tolist()
    return encoded_text

In [88]:
tt_split['train'][0]

{'statement': 'I cheated on my now ex. I hid the fact that I was talking to two other people. I went on a few dates with one of them. Nothing sexual or physical ever happened with either, but I cheated through and through. I lied to my ex so many times, right to their face. Even now I have not come clean. I ruined a great relationship for no good reason and really hurt someone. I do not think I can ever live this down or forgive myself for what I did. I cheated',
 'Anxiety': False,
 'Bipolar': False,
 'Depression': False,
 'Normal': False,
 'Personality Disorder': False,
 'Stress': False,
 'Suicidal': True,
 '__index_level_0__': 23510}

In [83]:
encoded_data = tt_split.map(preprocessing, batched=True, remove_columns=tt_split['train'].column_names)

Map:   0%|          | 0/42144 [00:00<?, ? examples/s]

Map: 100%|██████████| 10537/10537 [00:01<00:00, 10189.99 examples/s]


In [84]:
encoded_data['train'][0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [85]:
tokenizer.decode(encoded_data['train'][0]['input_ids'])

'[CLS] i cheated on my now ex. i hid the fact that i was talking to two other people. i went on a few dates with one of them. nothing sexual or physical ever happened with either, but i cheated through and through. i lied to my ex so many times, right to their face. even now i have not come clean. i ruined a great relationship for no good reason and really hurt someone. i do not think i can ever live this down or forgive myself for what i did. i cheated [SEP]'

In [86]:
encoded_data['train'][0]['labels']

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]

In [87]:
[id2key[idx] for idx, label in enumerate(encoded_data['train'][0]['labels']) if label == 1.0]

['Suicidal']

In [89]:
encoded_data.set_format("torch")

In [90]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           num_labels=len(key_list),
                                                           id2label=id2key,
                                                           label2id=key2id)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')
